In [ ]:
import pygame
from pygame.locals import *
from pygame.math import Vector2
import random
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class NeuralNetwork:
    class NN:
        def __init__(self):
            self.layers = []

    class Layer:
        def __init__(self):
            self.neurons = []
            self.neuron_count = 0

    class Dendrite:
        def __init__(self, weight):
            self.weight = weight

    class Neuron:
        def __init__(self):
            self.dendrites = []
            self.dendrite_count = 0
            self.value = 0

    # Constructor to create the neural network
    def __init__(self, input_layer, hidden_layer, output_layer):
        random.seed()
        self.network = self.NN()
        nn_layers = [input_layer, hidden_layer, output_layer]
        self.network.layers = []

        for i in range(3):
            layer = self.Layer()
            layer.neuron_count = int(nn_layers[i])
            layer.neurons = []

            for _ in range(int(nn_layers[i])):
                neuron = self.Neuron()
                if i > 0:
                    neuron.dendrite_count = int(nn_layers[i - 1])
                    neuron.dendrites = [self.Dendrite(weight=self.get_random()) for _ in range(int(nn_layers[i - 1]))]
                layer.neurons.append(neuron)
            self.network.layers.append(layer)

    # Execute the neural network
    def run(self, input_array):
        for i in range(3):
            for j in range(self.network.layers[i].neuron_count):
                if i == 0:
                    self.network.layers[i].neurons[j].value = input_array[j]
                else:
                    self.network.layers[i].neurons[j].value = 0
                    for k in range(self.network.layers[i - 1].neuron_count):
                        self.network.layers[i].neurons[j].value += (
                            self.network.layers[i - 1].neurons[k].value *
                            self.network.layers[i].neurons[j].dendrites[k].weight)
                    self.network.layers[i].neurons[j].value = self.activation(self.network.layers[i].neurons[j].value)

        outputs = []
        for i in range(self.network.layers[-1].neuron_count):
            output = self.network.layers[-1].neurons[i].value
            outputs.append(max(-1, min(output, 1)))
        return outputs
    
    # Random values between 1 and -1
    def get_random(self):
        return random.uniform(-1, 1)

    # Get the number of dendrites in the network
    def get_num_dendrites(self):
        num_dendrites = 0
        for i in range(3):
            for _ in range(self.network.layers[i].neuron_count):
                if i > 0:
                    num_dendrites += self.network.layers[i - 1].neuron_count
        return num_dendrites

    # Get the connection weights of the network
    def get_weights(self):
        weights = []
        for i in range(3):
            for j in range(self.network.layers[i].neuron_count):
                if i > 0:
                    for k in range(self.network.layers[i - 1].neuron_count):
                        weights.append(self.network.layers[i].neurons[j].dendrites[k].weight)
        return weights
    
    # Set new connection weights
    def set_weights(self, weights):
        num_dendrites = 0
        for i in range(3):
            for j in range(self.network.layers[i].neuron_count):
                if i > 0:
                    for k in range(self.network.layers[i - 1].neuron_count):
                        self.network.layers[i].neurons[j].dendrites[k].weight = weights[num_dendrites]
                        num_dendrites += 1
    
    # Activation function sigmoid
    def activation(self, value):
        return 1 / (1 + np.exp(value * (-1)))
    
class GeneticAlgorithm:
    # Initialisations
    def __init__(self, crossover_chance, mutation_chance):
        self.generation = 0
        self.elitism_chance = 100 - crossover_chance
        self.crossover_chance = crossover_chance
        self.mutation_chance = mutation_chance
        self.next_generation = []
        self.fitness_list = []
        self.average_fitness = 0
        self.highest_fitness = 0
    
    # Select a bug as a parent based on parent_chance
    def selection(self):
        self.next_generation.sort(key=lambda b: b.parent_chance, reverse=True)
        rand = random.randint(0, 100)
        valid = False

        for n in self.next_generation:
            if n.parent_chance > rand:
                valid = True
                return n
            
        if valid == False:
            return self.selection()
    
    # Perform crossover operation to create new children
    def crossover(self, bugs, bounds):
        num_crossover = int(len(bugs) * (self.crossover_chance / 100))

        for _ in range(num_crossover):
            parent_a = self.selection()
            parent_b = self.selection()

            parent_a_weights = parent_a.nn.get_weights()
            parent_b_weights = parent_b.nn.get_weights()

            crossover_point = random.randint(0, len(parent_a_weights)-1)

            child_weights = parent_a_weights[:crossover_point] + parent_b_weights[crossover_point:]

            child = Bug(bounds)
            child.nn.set_weights(child_weights)
            self.next_generation.append(child)

    # Apply mutation only on children generated through crossover
    def mutation(self):
        num_children = int(len(self.next_generation) * (self.crossover_chance / 100))
        num_parent = len(self.next_generation) - num_children
        idx_children = list(range(num_parent, num_parent+num_children))

        for idx in idx_children:
            i = self.next_generation[idx]
            if random.randint(0, 100) < self.mutation_chance:
                weights = i.nn.get_weights()
                mutation_point = random.randint(0, i.nn.get_num_dendrites()-1)
                weights[mutation_point] = random.uniform(0, 1)
                i.nn.set_weights(weights)

    # Implement elitism to preserve top-performing bugs
    def elitism(self, bugs):
        bugs.sort(key=lambda b: b.fitness, reverse=True)
        num = int(len(bugs) * self.elitism_chance / 100)
        self.next_generation.extend(bugs[:num])

    # Calculate average and highest fitness
    def fitness(self, bugs):
        self.average_fitness = 0
        self.highest_fitness = 0

        for b in bugs:
            self.fitness_list.append(b.fitness)
            self.average_fitness += b.fitness
            if b.fitness > self.highest_fitness:
                self.highest_fitness = b.fitness
        self.average_fitness /= len(bugs)

        for b in bugs:
            if self.highest_fitness == 0:
                b.parent_chance = 100
            else:
                b.parent_chance = (b.fitness / self.highest_fitness) * 100

    # Generate the next generation
    def next(self, bugs, bounds):
        self.next_generation = []
        
        self.fitness(bugs)
        self.elitism(bugs)
        self.crossover(bugs, bounds)
        self.mutation()
        num_bugs = min(len(self.next_generation), len(bugs))
        for i in range(num_bugs):
            bugs[i] = self.next_generation[i]
            bugs[i].reset()
        self.generation += 1
        self.next_generation.clear()
        
        return bugs

class Food:
    def __init__(self, bounds):
        # Initialise the random number generator
        random.seed()
        # Generate random coordinates for the position of the food
        self.position = Vector2(random.randint(bounds.left, bounds.right), random.randint(bounds.top, bounds.bottom))

    # Display the food's image and position on the screen
    def set(self, screen, image):
        screen.blit(image, self.position)

class Bug:
    def __init__(self, bounds):
        # Initialise the random number generator
        random.seed()

        # Initialise attributes
        self.nn = NeuralNetwork(2, 20, 3)
        self.position = Vector2(random.randint(bounds.left, bounds.right), random.randint(bounds.top, bounds.bottom))
        self.angle = random.randint(0, 360)
        self.bounds = bounds
        self.fitness = 0
        self.speed = 0.0
        self.parent_chance = 0
        self.colour = (0, 0, 255)
        self.path = [] # bug's trajectory

    # Reset the angle and fitness for the bug
    def reset(self):
        random.seed()
        self.angle = random.randint(0, 360)
        self.fitness = 0

    def update(self, food):
        # Calculate positions of sensors
        dis_sensor = math.sqrt(BUG_SIZE**2 * 2) / 2
        left_sensor = self.position + Vector2(math.cos(math.radians(self.angle-135)) * dis_sensor, math.sin(math.radians(self.angle-135)) * dis_sensor)
        right_sensor = self.position + Vector2(math.cos(math.radians(self.angle-45)) * dis_sensor, math.sin(math.radians(self.angle-45)) * dis_sensor)
        
        # Calculate distances from sensors to food items
        left_to_food = self.get_distance(self.get_closest_food(food, self.position).position, left_sensor)
        right_to_food = self.get_distance(self.get_closest_food(food, self.position).position, right_sensor)
        center_to_food = self.get_distance(self.get_closest_food(food, self.position).position, self.position)

        dis_center = FOOD_SIZE/2 + BUG_SIZE/2
        #dis_center = math.sqrt((BUG_SIZE)**2 * 2) / 2 + FOOD_SIZE/2

        # If bug and food are collided
        if center_to_food < dis_center:
            # Update fitness and then produce new food
            self.fitness += 1
            food_position = self.get_closest_food(food, self.position).position
            food_position.x = random.randint(self.bounds.left, self.bounds.right)
            food_position.y = random.randint(self.bounds.top, self.bounds.bottom)
        
        # Calculate inputs for neural network
        input_data = [0.0] * 2
        if left_to_food > right_to_food:
            input_data[0] = 1
            input_data[1] = -1
        else:
            input_data[0] = -1
            input_data[1] = 1

        # Run the neural network
        output = self.nn.run(input_data)

        # Update bug's angle and speed based on neural network output
        if output[0] > output[1]:
            self.angle += output[0] * 4
            #self.angle += 5
        else:
            self.angle -= output[1] * 4
            #self.angle -= 5

        self.speed = output[2] * 2
        
        # Update bug's position
        self.position.x += math.cos(math.radians(self.angle - 90)) * self.speed
        self.position.y += math.sin(math.radians(self.angle - 90)) * self.speed
        
        # Wrapping around screen if reaching the boundaries
        if self.position.x < 0:
            self.position.x = SCREEN_X - BUG_SIZE
        elif self.position.x >= SCREEN_X:
            self.position.x = 0
        if self.position.y < 0:
            self.position.y = SCREEN_Y - BUG_SIZE
        elif self.position.y >= SCREEN_Y:
            self.position.y = 0
        
        # Update bug's colour based on fitness
        self.colour = self.get_colour()

        # Append current position to the trajectory
        self.path.append(self.position.copy())
    
    # Display bug's colour and position on screen
    def set(self, screen, image):
        bug_colour_surface = pygame.Surface((BUG_SIZE, BUG_SIZE))
        bug_colour_surface.fill(self.colour)
        blended_surface = pygame.Surface((BUG_SIZE, BUG_SIZE), pygame.SRCALPHA)
        blended_surface.blit(image, (0, 0))
        blended_surface.blit(bug_colour_surface, (0, 0), special_flags=pygame.BLEND_RGB_MULT)

        destination_rect = pygame.Rect(int(self.position.x), int(self.position.y), BUG_SIZE, BUG_SIZE)
        rotated_image = pygame.transform.rotate(blended_surface, -self.angle)
        rotated_rect = rotated_image.get_rect(center=destination_rect.center)

        origin = Vector2(BUG_SIZE, BUG_SIZE)
        screen.blit(rotated_image, rotated_rect.center - origin // 2)

    # Calculate bug's colour based on its fitness
    def get_colour(self): # blue
        max_fitness = 20
        fitness = max(0, min(max_fitness, self.fitness))

        value = int(255 - fitness * 255 / max_fitness)
        return (0, value, 255)

    # Find the closest food item to the bug
    def get_closest_food(self, food, start):
        closest_food = None
        closest_distance = SCREEN_X
        for f in food:
            distance = self.get_distance(start, f.position)
            if distance < closest_distance:
                closest_distance = distance
                closest_food = f
        return closest_food

    # Calculate Euclidean distance between two points
    def get_distance(self, a, b):
        c = a - b
        return math.sqrt(c.x ** 2 + c.y ** 2)
        
class Start:
    def __init__(self, bounds):
        bug_image = pygame.image.load("bug.png")
        self.bug_image = pygame.transform.scale(bug_image, (BUG_SIZE, BUG_SIZE))
        bug_bounds = pygame.Rect(0, 0, SCREEN_X-BUG_SIZE, SCREEN_Y-BUG_SIZE)
        self.bugs = [Bug(bug_bounds) for _ in range(BUG_NUMBER)]

        food_image = pygame.image.load("food.png")
        self.food_image = pygame.transform.scale(food_image, (FOOD_SIZE, FOOD_SIZE))
        food_bounds = pygame.Rect(0, 0, SCREEN_X-FOOD_SIZE, SCREEN_Y-FOOD_SIZE)
        self.food = [Food(food_bounds) for _ in range(FOOD_NUMBER)]

        self.ga = GeneticAlgorithm(CROSSOVER_RATE, MUTATION_RATE)
        self.ticks = 0
        self.font = pygame.font.SysFont(None, 24)
        self.graphs = [0.0] * 1000
        self.bounds = bounds
        self.generation_data = []
        self.avg_fitness_data = []
        self.sd_fitness_data = []
        self.avg_speed_data = []
        self.highest_fitness_data = []

        self.paused = False # Initialise the paused state
        self.show_path = True # Show the bug's trajectory
    
    def update(self):
        for bug in self.bugs:
            bug.update(self.food)
        
        self.ticks += 1
        if self.ticks % 1000 == 0:
            self.ticks = 0
            self.graphs[self.ga.generation] = self.ga.next(self.bugs, self.bounds)
            self.generation_data.append(self.ga.generation)
            self.avg_fitness_data.append(self.ga.average_fitness)
            self.sd_fitness_data.append(np.std(self.ga.fitness_list))
            avg_speeds = [bug.speed for bug in self.bugs]
            self.avg_speed_data.append(sum(avg_speeds) / len(avg_speeds))
            self.highest_fitness_data.append(self.ga.highest_fitness)

    def show(self, screen):
        for f in self.food: # food
            f.set(screen, self.food_image)
        for b in self.bugs: # bugs
            b.set(screen, self.bug_image)
            #if self.show_path and len(b.path) >= 2: # the bug's trajectory
            #    pygame.draw.lines(screen, b.colour, False, b.path)
            
        #if self.ticks == 0:
            #plt.pause(0.001)
            #plt.clf()

            #self.save_file(self.avg_fitness_data, 'avg_fitness_data_c50m20n20.txt')
            #self.save_file(self.avg_speed_data, 'avg_speed_data_c50m20n20.txt')
            #self.save_file(self.highest_fitness_data, 'highest_fitness_data_c50m20n20.txt')
            
        screen.blit(self.font.render("Generation: " + str(self.ga.generation), True, (0, 0, 0)), (10, 10))
        screen.blit(self.font.render("Tick: " + str(self.ticks), True, (0, 0, 0)), (10, 30))
        screen.blit(self.font.render("Crossover: " + str(self.ga.crossover_chance) + "%", True, (0, 0, 0)), (10, 50))
        screen.blit(self.font.render("Mutation: " + str(self.ga.mutation_chance) + "%", True, (0, 0, 0)), (10, 70))

    # Save data to a file
    def save_file(self, data, filename):
        with open(filename, 'w') as file:
            for d in data:
                file.write(str(d) + '\n')

SCREEN_X = 600
SCREEN_Y = 600
BUG_SIZE = 10
BUG_NUMBER = 10
FOOD_SIZE = 10
FOOD_NUMBER = 200
CROSSOVER_RATE = 50
MUTATION_RATE = 20

def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_X, SCREEN_Y))
    clock = pygame.time.Clock()
    bounds = pygame.Rect(0, 0, SCREEN_X, SCREEN_Y)
    start = Start(bounds)

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                # Pause / unpause the game
                if event.key == pygame.K_p:
                    start.paused = not start.paused
                    start.show_path = not start.show_path
        
        # Fill the screen if not paused
        if not start.paused:
            screen.fill((255, 255, 255))
        
        # Always keep the game on the screen
        start.show(screen)

        # Run if not paused
        if not start.paused:
            start.update()

        pygame.display.flip()
        clock.tick(1000)

    pygame.quit()

main()